In [ ]:
#@title Update Plotly to latest version
!pip install -U plotly

In [ ]:
%%capture

!rm *.csv*

#@title Download Scrapped Data from Inteli.Gente
!wget "https://github.com/Rafaelsoz/Pratica-Ciencia-Dados-II/raw/main/datasets/Dimens%C3%B5es/Econ%C3%B4mica.csv"
!wget "https://github.com/Rafaelsoz/Pratica-Ciencia-Dados-II/raw/main/datasets/Dimens%C3%B5es/Capacidades%20Institucionais.csv"
!wget "https://github.com/Rafaelsoz/Pratica-Ciencia-Dados-II/raw/main/datasets/Dimens%C3%B5es/Meio%20Ambiente.csv"
!wget "https://github.com/Rafaelsoz/Pratica-Ciencia-Dados-II/raw/main/datasets/Dimens%C3%B5es/Sociocultural.csv"

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.colors as mcolors
from matplotlib import pyplot as plt

sns.set_style("darkgrid")

In [ ]:
# Load data
ECO = pd.read_csv("Econômica.csv").iloc[:, 1:]
CI = pd.read_csv("Capacidades Institucionais.csv").iloc[:, 1:]
MA = pd.read_csv("Meio Ambiente.csv").iloc[:, 1:]
SC = pd.read_csv("Sociocultural.csv").iloc[:, 1:]

# Target variable
level = ECO['Nivel']

In [ ]:
#@markdown Percebemos que todas as dimensões possuem valores nulos (NaNs) em várias cidades. Isso acontece por que a plataforma Inteli.Gente não encontrou dados sobre aquele atributo para aquele município. Na análise da plataforma, os valores nulos são considerados 0, então vamos substituir todos os valores nulos por zero nos dados.
ECO = ECO.fillna(0)
CI = CI.fillna(0)
MA = MA.fillna(0)
SC = SC.fillna(0)

In [ ]:
#@title Seleciona Apenas Colunas Numéricas e Remove Colunas com Valores Constantes

# Converte "Coleta seletiva de resíduos no município" de booleano para inteiro
ECO["Coleta seletiva de resíduos no município"] = ECO["Coleta seletiva de resíduos no município"].astype("int64")

# Select only numerical values
ECO = ECO[ECO.columns[(ECO.dtypes == "float64") | (ECO.dtypes == "int64")]]
CI = CI[CI.columns[(CI.dtypes == "float64") | (CI.dtypes == "int64")]]
MA = MA[MA.columns[(MA.dtypes == "float64") | (MA.dtypes == "int64")]]
SC = SC[SC.columns[(SC.dtypes == "float64") | (SC.dtypes == "int64")]]


# Remove 'Codigo', 'Nome' & 'Nivel'
ECO = ECO.iloc[:, 2:]
CI = CI.iloc[:, 2:]
MA = MA.iloc[:, 2:]
SC = SC.iloc[:, 2:]

# Remove constant valued columns
ECO = ECO[ECO.columns[ECO.nunique() > 1]]
CI = CI[CI.columns[CI.nunique() > 1]]
MA = MA[MA.columns[MA.nunique() > 1]]
SC = SC[SC.columns[SC.nunique() > 1]]

In [ ]:
#@title Multi Scatter Plot 1-D
import pandas as pd
import plotly.graph_objects as go

def multi_scatter_1d(df: pd.DataFrame,
                     target: pd.Series,
                     colormap: str = "Bluered"):

  """
  Generates a normalized one-dimensional scatterplot for each column in the DataFrame.

  Parameters
  ----------
  df : pd.DataFrame
      A multi-dimensional DataFrame containing the data to be plotted.

  target : pd.Series
      The target data, either discrete or continuous, used to colorize the scatterplot.

  colormap: str
      Colormap name to be used when setting the color scale.
  """

  # Apply normalization to data to be inside [0, 1] range
  data = (df - df.min()) / (df.max() - df.min())

  # Array to store the plot data
  values = []

  for idx, column in enumerate(data.columns):

    # Get the unique values and its frequency
    row, indices, counts = np.unique(data.iloc[:, idx], return_counts=True, return_index=True)

    # Sort the target values by the unique values order
    target_data = target[indices]

    # Add the index of each column to it's name
    column_text = f"({idx}) {column}"

    # Create a DataFrame with metadata for the current column
    row = pd.DataFrame(zip([column_text] * row.size, row, counts, target_data),
                      columns=["column", "value", "count", "target"])

    # Set the point colors with the most frequent target value
    row["color"] = row["value"].apply(lambda x: target[data[column] == x].mode()).iloc[:, 0]

    # Append current column to values array
    values.append(row)

  # Concatenate all columns
  values = pd.concat(values)

  fig = go.Figure(
    go.Scatter(
      x=values["column"],
      y=values["value"],
      mode='markers',
      marker=dict(
          size=values["count"].apply(lambda x: x ** (1/10)) * 10,
          color=values["color"],
          showscale=True,
          colorscale=colormap,
          line=dict(width=0),
          colorbar=dict(title=dict(text="Escala do Valor Alvo", side="right"))
      ),
      text=values["color"].apply(lambda x: f"Valor Alvo: {x}")
    )
  )

  fig.update_layout(
    title=dict(
      text="Distribuição dos Valores",
      subtitle=dict(
          text=f"Distribuição dos Valores de Cada Coluna do DataFrame ({df.shape[1]} Colunas)",
          font=dict(color="gray", size=13),
      ),
    ),
    xaxis_title="Colunas",
    yaxis_title="Valores Normalizados",
    template="plotly_white",
    margin = dict(t=60, b=40, l=40, r=20), # top, bottom, left, right
    width=1000,
    height=500,
  )

  fig.update_xaxes(showticklabels=False)

  fig.show()

In [ ]:
#@title Compute Meta-Features
# data = (ECO - ECO.min()) / (ECO.max() - ECO.min())
# target = level

def compute_metafeatures(data: pd.DataFrame,
                         target: pd.Series):

  # Apply normalization to data to be inside [0, 1] range
  data = (data - data.min()) / (data.max() - data.min())

  # Array to store the plot data
  values = []

  for idx, column in enumerate(data.columns):

    # Get the unique values and its frequency
    row, indices, counts = np.unique(data.iloc[:, idx], return_counts=True, return_index=True)

    # Sort the target values by the unique values order
    target_data = target[indices]

    # Add the index of each column to it's name
    # column_text = f"({idx}) {column}"
    column_text = column

    # Create a DataFrame with metadata for the current column
    row = pd.DataFrame(zip([column_text] * row.size, row, counts, target_data),
                      columns=["column", "value", "count", "target"])

    # Set the point colors with the most frequent target value
    row["color"] = row["value"].apply(lambda x: target[data[column] == x].mode()).iloc[:, 0]

    # Append current column to values array
    values.append(row)

  # Concatenate all columns
  values = pd.concat(values)

  nunique = data.nunique()

  mean_value = data.mean()
  median_value = data.median()
  dist_shift = mean_value - median_value

  mean_size = (data.count() - data.nunique()) / data.nunique()

  max_target = values.groupby(by='column')[['value', 'target']].apply(lambda x: x.iloc[x['value'].argmax()]['target'])
  min_target = values.groupby(by='column')[['value', 'target']].apply(lambda x: x.iloc[x['value'].argmin()]['target'])

  q25 = data.quantile(0.25)
  q50 = data.quantile(0.50)
  q75 = data.quantile(0.75)

  return nunique, mean_value, median_value, dist_shift, mean_size, max_target, min_target, q25, q50, q75

In [ ]:
#@title Função para Organizar Colunas Seguindo o Método 3
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import silhouette_score, make_scorer

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import warnings

def sort_columns_kmeans(metafeatures: pd.DataFrame):

  X = metafeatures

  kmeans = Pipeline([
    ('Padronização', StandardScaler()),
    ('KMeans', KMeans(n_clusters=2, random_state=0, n_init="auto")),
  ])


  with warnings.catch_warnings():
      warnings.simplefilter("ignore")

      clf = GridSearchCV(estimator=kmeans,
                        param_grid={'KMeans__n_clusters': np.arange(2, 10, 1)},
                        scoring=make_scorer(silhouette_score)).fit(X, X)

  n_clusters = clf.best_params_['KMeans__n_clusters']

  labels = Pipeline([
    ('Padronização', StandardScaler()),
    ('KMeans', KMeans(n_clusters=n_clusters, random_state=0, n_init="auto")),
  ]).fit(X)['KMeans'].labels_

  columns_order = pd.DataFrame(labels, index=X.index, columns=['Kmeans']).sort_values(by='Kmeans').index

  return columns_order

In [ ]:
#@title Dimensão Econômica
metafeatures = compute_metafeatures(ECO, level)
X = pd.concat(metafeatures, axis=1)

columns_order = sort_columns_kmeans(metafeatures=X)

multi_scatter_1d(ECO[columns_order], level)

In [ ]:
import numpy as np

def create_vector(n: int = 100, u: float = 0.5):

    # Calcula o número de valores unicos do vetor
    u = int(n * u)

    # Gera u valores únicos
    unique_values = np.random.choice(range(1, u + 1), u, replace=False)

    # Repete esses valores para preencher o vetor de tamanho n
    vector = np.random.choice(unique_values, n, replace=True)

    return vector

In [ ]:
import zlib
from tqdm.notebook import tqdm

def r(x, zx):
  return 1 - (x - zx) / x

def compute_r(A: np.array):
    x = A.size * A.itemsize
    zx = len(zlib.compress(A))

    return r(x, zx)

runs = {}

for n in tqdm([100, 500, 1000]):

  uvalues = np.linspace(0.01, 1, n)
  rvalues = np.array([compute_r(create_vector(n, u)) for u in uvalues])

  runs[n] = (uvalues, rvalues)

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from scipy import optimize

def f(x, b0, b1, b2, b3):
	""" Função para ajustar aos dados """
	return b0 * (1 - np.exp(-b1 * x + b2)) + b3

x = runs[1000][0]
y = runs[1000][1]

params, covariance = optimize.curve_fit(
	f=f, # Função para ajustar aos dados
	xdata=x, # Valores de entrada da função
	ydata=y, # Valores de saída da função
	p0=[0.1, 0.1, .1, .1] # Chute inicial para os parâmetros da função
)

print("Parâmetros Ajustados:", params)
print("Matriz de Covariância dos Parâmetros:", covariance)

# Out[1]: Parâmetros Ajustados: [10. 5.]
# Out[2]: Matriz de Covariância dos Parâmetros: [[ 0. -0.] [-0. 0.]]

Parâmetros Ajustados: [ 0.19860706  5.26508215 -0.4357783   0.06551951]
Matriz de Covariância dos Parâmetros: [[ 2.72868336e+07  1.75669045e+01 -1.36877790e+08 -2.72868338e+07]
 [ 1.75669045e+01  3.87636553e-03 -8.81199279e+01 -1.75669176e+01]
 [-1.36877790e+08 -8.81199279e+01  6.86614268e+08  1.36877791e+08]
 [-2.72868338e+07 -1.75669176e+01  1.36877791e+08  2.72868340e+07]]


In [ ]:
from scipy.interpolate import InterpolatedUnivariateSpline

spl = InterpolatedUnivariateSpline(x=runs[100][0], y=runs[100][1])
spl.set_smoothing_factor(0.0)

In [ ]:
#@markdown A porcentagem de valores únicos no vetor de entrada possui influência sob o valor final do fator de aleatoriedade. Isso causa problemas ao compararmos variáveis que possuem quantidades de valores únicos diferentes.

#@markdown Para remover esse víes da análise, ajustei uma função ao comportamento de $r(x, Z(x))$ em função do número de valores únicos em $x$, e defini os resíduos como: $r(x, Z(x)) - f(u)$, sendo $u$ a proporção de valores únicos no vetor de entrada.

import plotly.graph_objects as go

fig = go.Figure(
	[go.Scatter(
		x=runs[run][0],
		y=runs[run][1],
    name=run
	) for run in runs.keys()] +
  [
    go.Scatter(
        x=runs[run][0],
        # y=runs[run][1] - f(runs[run][0], *params),
        y=runs[run][1] - spl(runs[run][0]),
        name=f"Resíduos (n={run})"
    ) for run in runs.keys()] # +
  # [
  #   go.Scatter(
  #       x=runs[1000][0],
  #      # y=f(x, *params),
  #       y=spl(runs[1000][0]),
  #       name="Curva Ajustada"
  #   )
  # ]
)

fig.update_layout(
	title="Comportamento de r(x, zx) em Função do Número de Valores Únicos",
	xaxis_title="Porcentagem de Valores Únicos no Vetor",
	yaxis_title="Fator de Aleatoriedade",
	template="plotly_white",
	margin = dict(t=60, b=20, l=20, r=20), # top, bottom, left, right
	width=800,
	height=400,
  xaxis=dict(tickformat=',.0%', range=[0, 1])
)

# Adiciona linhas nos eixos da figura
fig.update_xaxes(showline=True, linewidth=.5, linecolor='grey')
fig.update_yaxes(showline=True, linewidth=.5, linecolor='grey')

fig.show()

In [ ]:
#@title Calcular Métricas para Análise de Aleatoriedade dos Atributos
import zlib
from scipy.stats import entropy

def f(u):
	""" Correção do víes do Fator de Aleatoriedade """
	return 0.15 * (1 - np.exp(-5.25 * u - 0.15)) + 0.1

def r(x, zx, u):
  return 1 - (x - zx) / x - spl(u)

def compute_entropy(vector: np.array,
                    target: np.array = None,
                    bins: int = 1000):

  # Compute the histogram bin counts from vector values
  hist, _ = np.histogram(vector, bins=bins)

  if target is not None:
    target_hist, _ = np.histogram(target, bins=bins)
    return entropy(hist, target_hist + 1)

  # Compute the Shannon entropy from histogram bin counts
  return entropy(hist)

metrics = []

for idx, column in enumerate(columns_order):
  A = ECO.loc[:, column].to_numpy()

  # Tamanho do Vetor em Bytes
  x = A.size * A.itemsize

  # Tamanho do Vetor Comprimido em Bytes
  zx = len(zlib.compress(A))

  # Porcentagem de Valores Únicos no Vetor
  u = np.unique(A).size / A.size

  # Fator de Aleatoriedade
  r_hat = r(x, zx, u)

  # Entropia de Shannon do Vetor
  e = compute_entropy(A)

  # Divergência de KL
  k = compute_entropy(A, level)

  metrics.append((f"({idx}) {column}", x, zx, u, r_hat, e, k))

metrics = pd.DataFrame(metrics, columns=['Nome', 'Tamanho (bytes)', 'Tamanho Zip (Bytes)',
                                         '% Valores Unicos', 'Aleatoriedade (r)', 'Entropia', 'KL'])

In [ ]:
metrics

,Nome,Tamanho (bytes),Tamanho Zip (Bytes),% Valores Unicos,Aleatoriedade (r),Entropia,KL
0,(0) Números de estações rádio base,44560,8657,0.046140,0.075705,3.313668,5.248805
1,(1) Escala de acesso a banda larga fixa de alt...,44560,42182,0.998923,0.707451,3.616966,4.374472
2,(2) Escala de acesso a banda larga móvel,44560,16256,0.756194,0.129404,3.393077,4.785704
3,(3) Percentual de domicílios com população viv...,44560,3784,0.105027,-0.072869,0.734299,3.445388
4,(4) Escala de acesso a banda larga fixa,44560,15058,0.504668,0.108025,3.340492,4.752045
5,(5) Dados abertos da gestão municipal,44560,2750,0.001975,0.103709,1.278135,7.059091
6,(6) Rede de tecnologia interligando os equipam...,44560,1453,0.000718,0.086685,0.616063,5.462505
7,(7) Sistema de informação geográfica da prefei...,44560,3479,0.001616,0.123471,1.759136,3.730043
8,(8) Ciclomobilidade na cidade,44560,1873,0.000898,0.094354,0.793860,2.310422
9,(9) Serviços e soluções inteligentes para mobi...,44560,1356,0.001257,0.079269,0.536269,3.417839


In [ ]:
#@title Relação entre Entropia de Shannon e Fator de Aleatoriedade

import plotly.graph_objects as go

# Calcula a correlação entre Fator de Aleatoriedade e Entropia
corr = metrics[['Aleatoriedade (r)', 'Entropia']].corr(method="spearman").iloc[0,1]

fig = go.Figure(
	go.Scatter(
     x=metrics['Aleatoriedade (r)'],
     y=metrics['Entropia'],
     text=metrics['Nome'],
     # hoverinfo="text+x+y",
     hovertemplate = '%{text}<br>Fator de Aleatoriedade: %{x:.4f}<br>Entropia de Shannon: %{y:.4f}<extra></extra>',
     mode="markers",
     marker_color="#05445E"
  )
)

fig.update_layout(

	title=dict(
		text="Relação entre Entropia de Shannon e Fator de Aleatoriedade",
		subtitle=dict(
          text=f"A Correlação de Spearman entre o Fator de Aleatoriedade e a Entropia de Shannon é de {corr:.4f}",
          font=dict(color="gray", size=13),
        ),
    ),

	xaxis_title="Fator de Aleatoriedade baseado em Compressão",
	yaxis_title="Entropia de Shannon",
	template="plotly_white",

	# Top, Bottom, Left, Right
	margin = dict(t=100, b=20, l=20, r=20),

	width=800,
	height=400,
)

# Adiciona linhas nos eixos da figura
fig.update_xaxes(showline=True, linewidth=.5, linecolor='grey')
fig.update_yaxes(showline=True, linewidth=.5, linecolor='grey')

fig.show()

- Selecionar Atributos em que as duas métricas consideram pouco aleatórios (bem comportados).

- Anotar atributos selecionados

- Comportamento das métricas converge quando usamos número de bins maior que 200

- Analisar métricas bem comportadas com o gráfico de multi scatter plot

- Analisar correlação par a par para encontrar métricas com informação reduntante

- Filtrar apenas métricas bem comportadas e pouco correlacionadas

- Modelar comportamento do atributo alvo usando os atributos selecionados e uma regressão linear